**Model Creation and Training**

In [0]:
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
%matplotlib inline

In [2]:
from google.colab import files
uploaded = files.upload()

Saving OnlineNewsPopularity.csv to OnlineNewsPopularity.csv


In [36]:
import io
data = pd.read_csv(io.BytesIO(uploaded['OnlineNewsPopularity.csv']))
display(data.head())

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [0]:
# Get the statistics of original target attribute
popularity_raw = data[data.keys()[-1]]
popularity_raw.describe()
# Encode the label by threshold 1400
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
popular_label = pd.Series(label_encoder.fit_transform(popularity_raw>=1400))

In [38]:
# Get the features from dataset
data.drop(data.iloc[:, 46:59], inplace=True, axis=1)
data.drop(data.iloc[:, 13:45], inplace=True, axis=1)
data.drop(data.iloc[:, 0:12], inplace=True, axis=1)
features_raw = data.drop([data.keys()[-1]], axis=1)
display(features_raw.head())

,num_keywords,global_sentiment_polarity,abs_title_sentiment_polarity
0,5.0,0.092562,0.187500
1,4.0,0.148948,0.000000
2,6.0,0.323333,0.000000
3,7.0,0.100705,0.000000
4,7.0,0.281003,0.136364


In [39]:
# Normalize the numerical features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical = [' num_keywords',' global_sentiment_polarity',' abs_title_sentiment_polarity']
features_raw[numerical] = scaler.fit_transform(data[numerical])
display(features_raw.head(n = 1))

,num_keywords,global_sentiment_polarity,abs_title_sentiment_polarity
0,0.444444,0.433591,0.1875


In [40]:
display(features_raw.head())

,num_keywords,global_sentiment_polarity,abs_title_sentiment_polarity
0,0.444444,0.433591,0.187500
1,0.333333,0.483864,0.000000
2,0.555556,0.639345,0.000000
3,0.666667,0.440851,0.000000
4,0.666667,0.601604,0.136364


In [41]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

estimator = AdaBoostClassifier(random_state=0)
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(features_raw, popular_label)
selector.ranking_

estimator_LR = LogisticRegression(random_state=0)
selector_LR = RFECV(estimator_LR, step=1, cv=5)
selector_LR = selector_LR.fit(features_raw, popular_label)
selector_LR.ranking_

estimator_RF = RandomForestClassifier(random_state=0)
selector_RF = RFECV(estimator_RF, step=1, cv=5)
selector_RF = selector_RF.fit(features_raw, popular_label)
selector_RF.ranking_

array([1, 1, 1])

In [42]:
# Split data into training and testing sets
from sklearn.metrics import accuracy_score, fbeta_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split

features_ADA = features_raw[features_raw.columns.values[selector.ranking_==1]]
features_LR = features_raw[features_raw.columns.values[selector_LR.ranking_==1]]
features_RF = features_raw[features_raw.columns.values[selector_RF.ranking_==1]]


X_train_ADA, X_test_ADA, y_train_ADA, y_test_ADA = train_test_split(features_ADA, popular_label, test_size = 0.1, random_state = 0)

X_train_LR, X_test_LR, y_train_LR, y_test_LR = train_test_split(features_LR, popular_label, test_size = 0.1, random_state = 0)

X_train_RF, X_test_RF, y_train_RF, y_test_RF = train_test_split(features_RF, popular_label, test_size = 0.1, random_state = 0)

print ("Training set has {} samples.".format(X_train_ADA.shape[0]))
print ("Testing set has {} samples.".format(X_test_ADA.shape[0]))

Training set has 35679 samples.
Testing set has 3965 samples.


In [0]:
def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    start = time() # Get start time
    learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() # Get end time

    results['train_time'] = end-start
        
    # Get predictions on the first 4000 training samples
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:4000])
    end = time() # Get end time
    
    # Calculate the total prediction time
    results['pred_time'] = end-start
            
    # Compute accuracy on the first 4000 training samples
    results['acc_train'] = accuracy_score(y_train[:4000],predictions_train)
        
    # Compute accuracy on test set
    results['acc_test'] = accuracy_score(y_test,predictions_test)
    
    # Compute F-score on the the first 4000 training samples
    results['f_train'] = fbeta_score(y_train[:4000],predictions_train,beta=1)
        
    # Compute F-score on the test set
    results['f_test'] = fbeta_score(y_test,predictions_test,beta=1)
    
    # Compute AUC on the the first 4000 training samples
    results['auc_train'] = roc_auc_score(y_train[:4000],predictions_train)
        
    # Compute AUC on the test set
    results['auc_test'] = roc_auc_score(y_test,predictions_test)
       
    # Success
    print ("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
    print ("{} with accuracy {}, F1 {} and AUC {}.".format(learner.__class__.__name__,\
          results['acc_test'],results['f_test'], results['auc_test']))   
    # Return the results
    return results

In [44]:
# Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
# Initialize the three models
clf_A = AdaBoostClassifier(random_state=0)
clf_B = LogisticRegression(random_state=0,C=1.0)
clf_C = RandomForestClassifier(random_state=0)

# Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_1 = int(X_train_ADA.shape[0]*0.01)
samples_10 = int(X_train_ADA.shape[0]*0.1)
samples_100 = X_train_ADA.shape[0]

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        if clf == clf_A:
            results[clf_name][i] = \
            train_predict(clf, samples, X_train_ADA, y_train_ADA, X_test_ADA, y_test_ADA)
        elif clf == clf_B:
            results[clf_name][i] = \
            train_predict(clf, samples, X_train_LR, y_train_LR, X_test_LR, y_test_LR)
        else:
            results[clf_name][i] = \
            train_predict(clf, samples, X_train_RF, y_train_RF, X_test_RF, y_test_RF)


AdaBoostClassifier trained on 356 samples.
AdaBoostClassifier with accuracy 0.5397225725094578, F1 0.6102925475122785 and AUC 0.531909675154826.
AdaBoostClassifier trained on 3567 samples.
AdaBoostClassifier with accuracy 0.5440100882723834, F1 0.6153191489361702 and AUC 0.5359903280592908.
AdaBoostClassifier trained on 35679 samples.
AdaBoostClassifier with accuracy 0.5586380832282472, F1 0.6430028559771521 and AUC 0.5480667342481491.
LogisticRegression trained on 356 samples.
LogisticRegression with accuracy 0.5250945775535939, F1 0.6885028949545079 and AUC 0.49978559800173766.
LogisticRegression trained on 3567 samples.
LogisticRegression with accuracy 0.5533417402269861, F1 0.6488201467380528 and AUC 0.5409493786806101.
LogisticRegression trained on 35679 samples.
LogisticRegression with accuracy 0.5523329129886507, F1 0.649417341497136 and AUC 0.5396815881614384.
RandomForestClassifier trained on 356 samples.
RandomForestClassifier with accuracy 0.519546027742749, F1 0.55396862561

In [0]:
def gridsearch(clf,parameters,X_train, y_train, X_test, y_test):
    scorer = make_scorer(roc_auc_score)
    grid_obj = GridSearchCV(clf, parameters, scoring=scorer)

# Fit the grid search object to the training data and find the optimal parameters
    grid_fit = grid_obj.fit(X_train, y_train)

# Get the estimator
    best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
    predictions = (clf.fit(X_train, y_train)).predict(X_test)
    best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
    print (clf.__class__.__name__)
    print ("Unoptimized model\n------")
    print ("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
    print ("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions,beta=1)))
    print ("AUC on testing data: {:.4f}".format(roc_auc_score(y_test, predictions)))
    print ("\nOptimized Model\n------")
    print ("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
    print ("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta=1)))
    print ("Final AUC on the testing data: {:.4f}".format(roc_auc_score(y_test, best_predictions)))

    print (best_clf)

In [0]:
# Do the grid search for hyperparameters
from sklearn.metrics import make_scorer 
from sklearn.model_selection import GridSearchCV
parameters_RF = {"n_estimators": [10,20,50,100,250,500]}
parameters_LR = {"penalty": ['l1','l2'],
              "C": [0.1,0.5,1.,2.,2.5,5]}
parameters_ADA = {"n_estimators": [100,200,300,400],
              "learning_rate": [0.1,0.5,1]}

In [47]:
# Grid search for Adaboost
gridsearch(clf_A,parameters_ADA,X_train_ADA, y_train_ADA, X_test_ADA, y_test_ADA)

AdaBoostClassifier
Unoptimized model
------
Accuracy score on testing data: 0.5586
F-score on testing data: 0.6430
AUC on testing data: 0.5481

Optimized Model
------
Final accuracy score on the testing data: 0.5596
Final F-score on the testing data: 0.6434
Final AUC on the testing data: 0.5492
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=400, random_state=0)


In [48]:
# Grid search for logistic regression
gridsearch(clf_B,parameters_LR,X_train_LR, y_train_LR, X_test_LR, y_test_LR)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_se

LogisticRegression
Unoptimized model
------
Accuracy score on testing data: 0.5523
F-score on testing data: 0.6494
AUC on testing data: 0.5397

Optimized Model
------
Final accuracy score on the testing data: 0.5518
Final F-score on the testing data: 0.6487
Final AUC on the testing data: 0.5392
LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [49]:
# Grid search for RF
gridsearch(clf_C,parameters_RF,X_train_RF, y_train_RF, X_test_RF, y_test_RF)

RandomForestClassifier
Unoptimized model
------
Accuracy score on testing data: 0.5130
F-score on testing data: 0.5427
AUC on testing data: 0.5110

Optimized Model
------
Final accuracy score on the testing data: 0.5148
Final F-score on the testing data: 0.5423
Final AUC on the testing data: 0.5130
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


In [50]:
# Run the classifier with refined hyperparameters
clf_A = AdaBoostClassifier(random_state=0,learning_rate=1,n_estimators=400)
clf_B = LogisticRegression(random_state=0, C=2.0)
clf_C = RandomForestClassifier(random_state=0, n_estimators=20)

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        if clf == clf_A:
            results[clf_name][i] = \
            train_predict(clf, samples, X_train_ADA, y_train_ADA, X_test_ADA, y_test_ADA)
        elif clf == clf_B:
            results[clf_name][i] = \
            train_predict(clf, samples, X_train_LR, y_train_LR, X_test_LR, y_test_LR)
        else:
            results[clf_name][i] = \
            train_predict(clf, samples, X_train_RF, y_train_RF, X_test_RF, y_test_RF)

AdaBoostClassifier trained on 356 samples.
AdaBoostClassifier with accuracy 0.5273644388398486, F1 0.5619448340345957 and AUC 0.524711073856833.
AdaBoostClassifier trained on 3567 samples.
AdaBoostClassifier with accuracy 0.5435056746532156, F1 0.6014971378247469 and AUC 0.5375098910618472.
AdaBoostClassifier trained on 35679 samples.
AdaBoostClassifier with accuracy 0.5596469104665825, F1 0.6433823529411765 and AUC 0.5491550699121424.
LogisticRegression trained on 356 samples.
LogisticRegression with accuracy 0.5263556116015132, F1 0.6885572139303482 and AUC 0.5012421541112891.
LogisticRegression trained on 3567 samples.
LogisticRegression with accuracy 0.5525851197982345, F1 0.6452 and AUC 0.5407419916198282.
LogisticRegression trained on 35679 samples.
LogisticRegression with accuracy 0.551828499369483, F1 0.6487448112275154 and AUC 0.5392271477570312.
RandomForestClassifier trained on 356 samples.
RandomForestClassifier with accuracy 0.5253467843631778, F1 0.5475961538461538 and AU

**Crawling news and dataset generation**

In [53]:
pip install newspaper3k

     |████████████████████████████████| 215kB 2.6MB/s 
     |████████████████████████████████| 194kB 48.5MB/s 
     |████████████████████████████████| 7.4MB 43.4MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=3e465d280ec736a10125ae03e904f945fc94908391c7bddc4a67eeeb01fddf93
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=5dc09e5865cbbf12a4c1bf823460ccbe0ca834485c8b936f086139b7d0e5fca6
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13539 sha256=b6747238a8c8314d9267e128b6b1744d6bd36e8dcdc7a458539c3938bafa8654
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7

In [0]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np


In [55]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
url = "https://timesofindia.indiatimes.com/world"
r = requests.get(url)

In [0]:
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.findAll('a', attrs = {'class':'w_img'})

In [0]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])

In [0]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

In [60]:
dataset=pd.DataFrame(df)
dataset.head()

,Title,Text,Summary,Keywords
0,France Lockdown Lift: France to outline plans ...,Nurse Sandrine poses as she works in a hotel o...,"Restaurants, cafes and cinemas will have to re...","[outline, week, lift, lawmakers, times, plans,..."
1,Covid-19: Testing won't 'be a problem' for reo...,"Apr 28, 2020, 08:40AM IST\n\nSource: AP\n\nThe...","Apr 28, 2020, 08:40AM ISTSource: APThe White H...","[trump, problem, virus, unveiling, house, bett..."
2,Covid-19: Some Georgia restaurants reopen for ...,"Apr 28, 2020, 08:43AM IST\n\nSource: AP\n\nWit...","Apr 28, 2020, 08:43AM ISTSource: APWith tables...","[wearing, dinein, customers, service, allowed,..."
3,'Very good idea' on Kim Jong Un's health: Pres...,"Apr 28, 2020, 08:39AM IST\n\nSource: AP\n\nNor...","Apr 28, 2020, 08:39AM ISTSource: APNorth Korea...","[health, trump, idea, reporters, korean, rumor..."
4,Covid-19: Mayor entertains residents after imp...,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...","Apr 27, 2020, 08:27AM ISTSource: APThe videos,...","[tshirt, residents, facebook, try, way, imposi..."


In [0]:
import time
import math
import re
from textblob import TextBlob
import pandas as pd

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import string

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

In [0]:
dataset['Text_clean'] = dataset['Text'].map(lambda x: re.sub('[^a-zA-Z0-9  . , : - _]', '', str(x)))
dataset['Title_clean'] = dataset['Title'].map(lambda x: re.sub('[^a-zA-Z0-9  . , : - _]', '', str(x)))

In [0]:
dataset.Text_clean = dataset.Text_clean.str.lower()
dataset.Title_clean = dataset.Title_clean.str.lower()

In [64]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 2.6MB/s 


In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [0]:
dataset['vader_sentiment_Text'] = dataset.Text_clean.apply(lambda x:analyser.polarity_scores(x))
dataset['vader_sentiment_Title'] = dataset.Title_clean.apply(lambda x:analyser.polarity_scores(x))

In [0]:
def sentiment_parser(x):
    if x['neg'] > 0:
        return x['neg']
    elif x['pos'] > 0:
        return x['pos']
    elif x['compound'] > 0:
      return x['compound']
    else:
      return 0

In [0]:
dataset['v_sentiment_Text'] = dataset['vader_sentiment_Text'].apply(lambda x:sentiment_parser(x))
dataset['v_sentiment_Title'] = dataset['vader_sentiment_Title'].apply(lambda x:sentiment_parser(x))

In [0]:
dataset.drop(dataset.iloc[:, 4:8], inplace=True, axis=1)
dataset.drop(dataset.iloc[:, 0:3], inplace=True, axis=1)

In [0]:
dataset['num_keywords'] = dataset['Keywords'].apply(lambda x: len(x))

In [71]:
dataset[' num_keywords'] = dataset['num_keywords']
dataset[' global_sentiment_polarity'] = dataset['v_sentiment_Text']
dataset[' abs_title_sentiment_polarity'] = dataset['v_sentiment_Title']
dataset.drop(dataset.iloc[:, 0:4], inplace=True, axis=1)
dataset.head()

,num_keywords,global_sentiment_polarity,abs_title_sentiment_polarity
0,13,0.075,0.000
1,14,0.039,0.184
2,12,0.062,0.000
3,15,0.073,0.242
4,15,0.031,0.143


In [72]:
# Normalize the numerical features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical = [' num_keywords',' global_sentiment_polarity',' abs_title_sentiment_polarity']
dataset[numerical] = scaler.fit_transform(dataset[numerical])
display(dataset.head(n = 1))

,num_keywords,global_sentiment_polarity,abs_title_sentiment_polarity
0,0.166667,0.356688,0.0


In [0]:
final_prediction_ADA = clf_A.predict(dataset)
final_prediction_LR = clf_B.predict(dataset.iloc[:, :2].values)
final_prediction_RF = clf_C.predict(dataset)

In [83]:
final_prediction_ADA

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1])

In [84]:
final_prediction_RF

array([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1])

In [85]:
final_prediction_LR

array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1])

**Here '1' denotes that the number of shares will be greater than 1400(our initial assumption for being viral) and so the article is going to be viral according to our hypothesis.**